In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


In [2]:
# Load datasets
customers = pd.read_csv(r"C:\Users\91976\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\91976\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\91976\Downloads\Transactions.csv")

In [3]:
# Merge datasets
transactions = transactions.merge(products, on='ProductID', how='left')
transactions = transactions.merge(customers, on='CustomerID', how='left')


In [4]:
# Feature Engineering
# Calculate total spent, average transaction value, and preferred category for each customer
customer_features = transactions.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    avg_transaction_value=('TotalValue', 'mean'),
    preferred_category=('Category', lambda x: x.value_counts().idxmax()),
    transaction_count=('TransactionID', 'count')
).reset_index()



In [5]:
# Encode preferred category
customer_features = pd.get_dummies(customer_features, columns=['preferred_category'])



In [6]:
# Normalize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(customer_features.iloc[:, 1:])



In [7]:
# Compute similarity matrix
similarity_matrix = cosine_similarity(features_scaled)



In [8]:
# Create recommendations for the first 20 customers
lookalike_map = {}
for i in range(20):
    customer_id = customer_features.iloc[i]['CustomerID']
    similarities = list(enumerate(similarity_matrix[i]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_lookalikes = [
        (customer_features.iloc[j]['CustomerID'], round(score, 2))
        for j, score in similarities[1:4]
    ]
    lookalike_map[customer_id] = top_lookalikes



In [9]:
# Create Lookalike.csv
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_map.keys(),
    'Lookalikes': [str(v) for v in lookalike_map.values()]
})
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike model completed. Recommendations saved to 'Lookalike.csv'.")

Lookalike model completed. Recommendations saved to 'Lookalike.csv'.
